<a href="https://colab.research.google.com/github/Lean-IQ/Production-Capacity-Planning-and-Value-Stream-Analysis/blob/main/Production_%26_Capacity_Planning_and_Value_Stream_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Production & Capacity Planning and Value Stream Analysis

The "Production / Capacity Planning and Value Stream Analysis" script is focused on aiding production and capacity planning by performing value stream analysis. It analyzes images of production processes using optical character recognition (OCR) to extract text, enabling the assessment of value streams. The primary goal is to explore various scheduling algorithms and evaluate their performance concerning maximum capacity utilization. By leveraging image processing techniques and OCR, the script aims to provide insights into production efficiency, bottlenecks, and potential improvements. It serves as a valuable tool for optimizing resource allocation and enhancing overall productivity in manufacturing environments.

[Lean-IQ](https://www.lean-iq.com)

**Dependencies**


*   opencv-python: For image processing tasks.
*   pytesseract: For optical character recognition (OCR) to extract text from images.


**Features**


*   Analyzes production processes from images.
*   Performs value stream analysis using OCR to extract text from images.


**Input Requirements**

*   Provide images of production processes for analysis.

**Output**

The script may generate various visualizations or reports based on the analyzed production processes.

**Configuration**

Adjust the script as needed to customize the analysis or output formats.

**Contributing**
Contributions are welcome! Please fork the repository, create a new branch, and submit a pull request. For major changes, open an issue first to discuss potential improvements.

**License**

This project is licensed under the MIT License - see the LICENSE file for details. (Annex)

In [ ]:
!pip install opencv-python
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pytesseract

# Creating a Randomized Data Set

**Dependencies**

*   google.colab: For mounting Google Drive (if running on Google Colab).


**Configuration**

*   Adjust the parameters in the script according to your requirements:
*   num_parts: Number of parts to generate.
*   num_production_steps: List of possible numbers of production steps per part.
*   num_machines: Total number of machines available on the shopfloor.
*   repetitive_parts_percentage: Percentage of parts that have repetitive production steps.
*   cycle_time_ranges: Dictionary defining the range of cycle times for each production step.


**Output**

The script generates a CSV file containing the random dataset of parts with the following columns:
Part ID: Identifier for each part.

*   Production Step: Step number in the production process.
*   Machine: Identifier for the machine used in each production step.
*   Cycle Time: Cycle time required for each production step.


**Contributing**

Contributions are welcome! Please fork the repository, create a new branch, and submit a pull request. For major changes, open an issue first to discuss potential improvements.

**License**

This project is licensed under the MIT License - see the LICENSE file for details. (Annex)

In [ ]:
# Create a Random List of Parts with given specifications for a pre-defined shopfloor environment

from google.colab import drive
import os
import csv
import random

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to save the CSV file
csv_file_path = '/content/drive/MyDrive/DATA/fertigungsfilm.csv'

# Generate a list of parts
num_parts = 2500
num_production_steps = [1, 2, 3]
num_machines = 11
repetitive_parts_percentage = 0
cycle_time_ranges = {
    1: (20, 22),
    2: (21, 23),
    3: (22, 24),
    4: (23, 25)
}

# Check if the directory exists, if not, create it
directory = os.path.dirname(csv_file_path)
if not os.path.exists(directory):
    os.makedirs(directory)

# Write the list of parts to a CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Part ID', 'Production Step', 'Machine', 'Cycle Time']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(num_parts):
        part_number = f'Part_{i + 1}'
        num_steps = random.choice(num_production_steps)
        machines = random.sample(range(1, num_machines + 1), num_steps)

        # Ensure different machines for each production step
        machines_unique = set()
        for _ in range(num_steps):
            machine = random.randint(1, num_machines)
            while machine in machines_unique:
                machine = random.randint(1, num_machines)
            machines_unique.add(machine)

        # Generate successor machine IDs
        successor_machines = {}
        for idx, machine in enumerate(sorted(machines_unique)):
            if idx == len(machines_unique) - 1:
                successor_machines[f'machine_{machine}'] = None
            else:
                successor_machines[f'machine_{machine}'] = f'machine_{sorted(machines_unique)[idx + 1]}'

        # Generate cycle times for each machine
        cycle_times = {}
        for machine in machines_unique:
            min_time, max_time = cycle_time_ranges[random.randint(1, len(cycle_time_ranges))]
            cycle_times[f'machine_{machine}'] = random.randint(min_time, max_time)

        # Write data to CSV
        for step, machine in enumerate(sorted(machines_unique), start=1):
            writer.writerow({'Part ID': part_number, 'Production Step': step, 'Machine': f'machine_{machine}', 'Cycle Time': cycle_times[f'machine_{machine}']})

print(f'CSV file saved at: {csv_file_path}')


# Calculating Key Performance Indicators (KPI)

**Dependencies**

*   pandas: For data manipulation and analysis.
*   google.colab: For mounting Google Drive (if running on Google Colab).


**Analysis Metrics**

The script calculates the following KPIs:


*   Total schedule time for each machine (in minutes).
*   Average cycle time for each machine (in minutes).
*   Standard deviation of cycle time for each machine.
*   Total planned capacity ratio for each machine.


**Input Data**

The script requires manufacturing data, typically generated from the previous script in the series, which contains information about parts, production steps, machines, and cycle times.

**Output**

The script generates a CSV file (fertigungsfilm_analysis.csv) containing the calculated KPIs for each machine. Additionally, it prints information about the analysis data and displays the analysis data table.

**Contributing**

Contributions are welcome! Please fork the repository, create a new branch, and submit a pull request. For major changes, open an issue first to discuss potential improvements.

**License**

This project is licensed under the MIT License - see the LICENSE file for details. (Annex)

In [ ]:
# Calculate the some KPI's based on the parts generated

import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the manufacturing data
manufacturing_data = pd.read_csv('/content/drive/MyDrive/DATA/fertigungsfilm.csv')

# Calculate the total schedule time for each machine (in minutes)
total_schedule_time = manufacturing_data.groupby('Machine')['Cycle Time'].sum()

# Calculate the average cycle time for each machine (in minutes)
average_cycle_time = manufacturing_data.groupby('Machine')['Cycle Time'].mean()

# Calculate the standard deviation of cycle time for each machine
cycle_time_std = manufacturing_data.groupby('Machine')['Cycle Time'].std()

# Number of shifts per day and days in the production freeze
number_of_shifts = 3
hours_per_shift = 8
number_of_days = 10  # 2 weeks with 5 workdays per week

# Calculate the total available time (in minutes)
total_available_time = number_of_shifts * hours_per_shift * number_of_days * 60  # Convert hours to minutes

# Calculate the total planned capacity ratio for each machine
planned_capacity_ratio = total_schedule_time / total_available_time

# Create a DataFrame for analysis
analysis_data = pd.DataFrame({
    'Machine': total_schedule_time.index,
    'Total Schedule Time': total_schedule_time.values,
    'Average Cycle Time': average_cycle_time.values,
    'Cycle Time Std': cycle_time_std.values,
    'Total Planned Capacity': planned_capacity_ratio.values
})

# Add a 'Total' row at the bottom
total_row = pd.DataFrame({
    'Machine': ['Total'],
    'Total Schedule Time': [analysis_data['Total Schedule Time'].sum()],
    'Average Cycle Time': [analysis_data['Average Cycle Time'].mean()],
    'Cycle Time Std': [analysis_data['Cycle Time Std'].mean()],
    'Total Planned Capacity': [analysis_data['Total Planned Capacity'].sum()]
})

analysis_data = pd.concat([analysis_data, total_row])

# Save the analysis data to a CSV file
analysis_data.to_csv('/content/drive/MyDrive/DATA/fertigungsfilm_analysis.csv', index=False)

# Print information about the analysis data
print("Analysis data saved to fertigungsfilm_analysis.csv")

# Print the analysis data
print("Analysis Data:")
print(analysis_data)


# HEIJUNKA Optimization

HEIJUNKA, or production leveling, is a lean manufacturing technique that aims to smooth and balance production over time to match customer demand. By distributing work evenly, it minimizes fluctuations, reduces waste, and improves efficiency. HEIJUNKA involves sequencing orders, balancing workloads, and maintaining flexibility to adapt to changes in demand, ultimately ensuring a consistent and efficient production flow.

**Dependencies**

*   pandas: For data manipulation and analysis.
*   google.colab: For mounting Google Drive (if running on Google Colab).


**Analysis Procedure**


*   Load manufacturing data and analysis data containing KPIs.
*   Calculate the upper and lower bounds for cycle time deviation based on the average cycle time and standard deviation.
*   Compare the cycle time of each part with the bounds and mark deviations.
*   Identify part IDs with marked deviations ('OUT').
*   Remove rows corresponding to parts marked as 'OUT' from the dataset.
*   Save the optimized manufacturing data to a new CSV file.

**Input Data**

The script requires manufacturing data (fertigungsfilm.csv) generated from previous scripts in the series. Additionally, it uses analysis data (fertigungsfilm_analysis.csv) containing KPIs calculated from the manufacturing data.


**Output**

The script generates a new CSV file (fertigungsfilm_optimized.csv) containing optimized manufacturing data with parts having cycle times within the specified tolerance range. It also prints the total number of unique part IDs in the optimized list.


**Contributing**

Contributions are welcome! Please fork the repository, create a new branch, and submit a pull request. For major changes, open an issue first to discuss potential improvements.


**License**

This project is licensed under the MIT License - see the LICENSE file for details. (Annex)

In [ ]:
# HEIJUNKA application, by focusing on parts with an average cycle time +/- XX standard deviation

import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the manufacturing data and analysis data
manufacturing_data = pd.read_csv('/content/drive/MyDrive/DATA/fertigungsfilm.csv')
analysis_data = pd.read_csv('/content/drive/MyDrive/DATA/fertigungsfilm_analysis.csv')

# Calculate the upper and lower bounds for cycle time deviation
average_cycle_time = analysis_data.loc[analysis_data['Machine'] == 'Total', 'Average Cycle Time'].values[0]
std_deviation = analysis_data.loc[analysis_data['Machine'] == 'Total', 'Cycle Time Std'].values[0]
tolerance = 2 * std_deviation
lower_bound = average_cycle_time - tolerance
upper_bound = average_cycle_time + tolerance

# Compare cycle time with bounds and mark deviations
manufacturing_data['Optimized'] = manufacturing_data.apply(
    lambda row: 'OUT' if row['Cycle Time'] < lower_bound or row['Cycle Time'] > upper_bound else '',
    axis=1
)

# Identify Part_IDs with 'OUT' marked
out_part_ids = manufacturing_data.loc[manufacturing_data['Optimized'] == 'OUT', 'Part ID'].unique()

# Remove rows with the same Part_IDs as those marked as 'OUT'
manufacturing_data = manufacturing_data[~manufacturing_data['Part ID'].isin(out_part_ids)]

# Save the optimized data to a new CSV file
manufacturing_data.to_csv('/content/drive/MyDrive/DATA/fertigungsfilm_optimized.csv', index=False)

# Print total number of unique part IDs in the list
total_unique_part_ids = manufacturing_data['Part ID'].nunique()
print(f"Total Unique Part IDs: {total_unique_part_ids}")
print("Optimized manufacturing data saved to fertigungsfilm_optimized.csv")


**Calculating Key Performance Indicators (KPI) for HEIJUNKA Optimization**

In [ ]:
# Calculate the some KPI's based on the parts generated after HEIJUNKA


import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the manufacturing data
manufacturing_data = pd.read_csv('/content/drive/MyDrive/DATA/fertigungsfilm_optimized.csv')

# Calculate the total schedule time for each machine (in minutes)
total_schedule_time = manufacturing_data.groupby('Machine')['Cycle Time'].sum()

# Calculate the average cycle time for each machine (in minutes)
average_cycle_time = manufacturing_data.groupby('Machine')['Cycle Time'].mean()

# Calculate the standard deviation of cycle time for each machine
cycle_time_std = manufacturing_data.groupby('Machine')['Cycle Time'].std()

# Number of shifts per day and days in the production freeze
number_of_shifts = 3
hours_per_shift = 8
number_of_days = 10  # 2 weeks with 5 workdays per week

# Calculate the total available time (in minutes)
total_available_time = number_of_shifts * hours_per_shift * number_of_days * 60  # Convert hours to minutes

# Calculate the total planned capacity ratio for each machine
planned_capacity_ratio = total_schedule_time / total_available_time

# Create a DataFrame for analysis
analysis_data = pd.DataFrame({
    'Machine': total_schedule_time.index,
    'Total Schedule Time': total_schedule_time.values,
    'Average Cycle Time': average_cycle_time.values,
    'Cycle Time Std': cycle_time_std.values,
    'Total Planned Capacity': planned_capacity_ratio.values
})

# Add a 'Total' row at the bottom
total_row = pd.DataFrame({
    'Machine': ['Total'],
    'Total Schedule Time': [analysis_data['Total Schedule Time'].sum()],
    'Average Cycle Time': [analysis_data['Average Cycle Time'].mean()],
    'Cycle Time Std': [analysis_data['Cycle Time Std'].mean()],
    'Total Planned Capacity': [analysis_data['Total Planned Capacity'].sum()]
})

analysis_data = pd.concat([analysis_data, total_row])

# Print the analysis data
print("Analysis Data:")
print(analysis_data)

# Shortest Processing Time (SPT) Model
The Shortest Processing Time (SPT) Model is a scheduling strategy used in manufacturing to prioritize jobs with the shortest processing times. This approach aims to minimize the total processing time and increase the efficiency of the production process by reducing waiting times and improving the flow of parts through the system. In this script, we implement the SPT model to schedule production steps within a defined time window, ensuring optimal machine utilization and part completion.

**Initialization**

The script mounts Google Drive and loads the manufacturing data.
Defines the time window and other parameters for scheduling.

**Data Preparation**

The manufacturing data is sorted by part ID, production step, and cycle time.


**Scheduling**

Each part's production steps are scheduled on machines based on the shortest processing time, considering machine availability and sequence.

**Output:**

The schedule for each machine is printed, showing the part ID, production step, start time, and end time.

Capacity utilization for each machine is calculated and displayed.
The total number of scheduled parts and the ratio of scheduled parts to total parts are printed.

**Configuration**

Adjust the script as needed to customize the analysis or output formats.

**Contributing**
Contributions are welcome! Please fork the repository, create a new branch, and submit a pull request. For major changes, open an issue first to discuss potential improvements.

**License**

This project is licensed under the MIT License - see the LICENSE file for details. (Annex)

In [ ]:
# SHORTEST PROCESSING TIME (SPT) MODEL

import pandas as pd

# Load manufacturing data
manufacturing_data = pd.read_csv('/content/drive/MyDrive/DATA/fertigungsfilm_optimized.csv')

# Define parameters
time_window_days = 10
shifts_per_day = 3
hours_per_shift = 8
minutes_per_shift = hours_per_shift * 60
total_time_minutes = time_window_days * shifts_per_day * minutes_per_shift

# Define due date calculation function
def calculate_due_date(start_time, cycle_time):
    return start_time + cycle_time

# Initialize schedule dictionary to store schedule per machine
schedule_per_machine = {}

# Sort manufacturing data by part ID, production step, and cycle time in ascending order
manufacturing_data_sorted = manufacturing_data.sort_values(by=['Part ID', 'Production Step', 'Cycle Time'])

# Initialize dictionary to track end times of manufacturing steps on each machine
end_times_per_machine = {machine: 0 for machine in manufacturing_data_sorted['Machine'].unique()}

# Initialize main clock
main_clock = 0

# Initialize variables to track scheduled parts
scheduled_parts = set()

# Schedule manufacturing steps considering sequence and machine availability
for _, row in manufacturing_data_sorted.iterrows():
    part_id = row['Part ID']
    machine = row['Machine']
    cycle_time = row['Cycle Time']
    step = row['Production Step']

    # Find suitable start time for the manufacturing step on the machine
    start_time = max(main_clock, end_times_per_machine[machine])

    # Calculate end time based on start time and cycle time
    end_time = calculate_due_date(start_time, cycle_time)

    # Check if the end time exceeds the total time window
    if end_time > total_time_minutes:
        break

    # Update schedule dictionary with the scheduled manufacturing step
    if machine not in schedule_per_machine:
        schedule_per_machine[machine] = []
    schedule_per_machine[machine].append((part_id, step, start_time, end_time))
    end_times_per_machine[machine] = end_time
    scheduled_parts.add(part_id)

    # Update main clock to the end time of the scheduled step
    main_clock = end_time

# Output the schedule per machine
for machine, schedule in schedule_per_machine.items():
    print(f"Schedule for Machine: {machine}")
    print("{:<15} {:<20} {:<20} {:<10}".format("Part ID", "Production Step", "Start Time", "End Time"))
    for part_id, step, start_time, end_time in schedule:
        print("{:<15} {:<20} {:<20} {:<10}".format(part_id, step, start_time, end_time))
    print()

# Calculate and display capacity utilization for each machine
print("\nCapacity Utilization per Machine:")
for machine, schedule in schedule_per_machine.items():
    machine_capacity = sum(end_time - start_time for _, _, start_time, end_time in schedule)
    utilization = (machine_capacity / total_time_minutes) * 100
    print(f"{machine}, {utilization:.2f}%")

# Calculate total number of parts scheduled and ratio
total_parts = len(manufacturing_data['Part ID'].unique())
scheduled_parts_count = len(scheduled_parts)
ratio = scheduled_parts_count / total_parts * 100 if total_parts != 0 else 0

print(f"\nTotal Parts Scheduled: {scheduled_parts_count}/{total_parts}")
print(f"Ratio of Scheduled Parts: {ratio:.2f}%")


#First Come First Serve (FCFS) Model
The First Come First Serve (FCFS) Model is a basic scheduling strategy where manufacturing steps are processed in the order they arrive, without prioritizing any specific criteria. This model ensures that parts are handled sequentially based on their arrival time, simplifying the scheduling process. The script below implements the FCFS model to schedule production steps, aiming to maximize machine utilization and track part completion.

**Initialization**

The script mounts Google Drive and loads the manufacturing data.
Defines the time window and other parameters for scheduling.

**Data Preparation**

The manufacturing data is sorted by part ID, production step, and cycle time.

**Scheduling**

Each part's production steps are scheduled on machines based on the FCFS model, considering machine availability and sequence.

**Output**

The schedule for each machine is printed, showing the part ID, production step, start time, and end time.

Capacity utilization for each machine is calculated and displayed.
The total number of scheduled parts and the ratio of scheduled parts to total parts are printed.

**Configuration**

Adjust the script as needed to customize the analysis or output formats.

**Contributing**
Contributions are welcome! Please fork the repository, create a new branch, and submit a pull request. For major changes, open an issue first to discuss potential improvements.

**License**

This project is licensed under the MIT License - see the LICENSE file for details. (Annex)

In [ ]:
# FIRST COME FIRST SERVE (FCFS) MODEL

import pandas as pd

# Load manufacturing data
manufacturing_data = pd.read_csv('/content/drive/MyDrive/DATA/fertigungsfilm_optimized.csv')

# Define parameters
time_window_days = 10
shifts_per_day = 3
hours_per_shift = 8
minutes_per_shift = hours_per_shift * 60
total_time_minutes = time_window_days * shifts_per_day * minutes_per_shift

# Define due date calculation function
def calculate_due_date(start_time, cycle_time):
    return start_time + cycle_time

# Initialize schedule dictionary to store schedule per machine
schedule_per_machine = {}

# Sort manufacturing data by part ID, production step, and cycle time in ascending order
manufacturing_data_sorted = manufacturing_data.sort_values(by=['Part ID', 'Production Step', 'Cycle Time'])

# Initialize dictionary to track end times of manufacturing steps on each machine
end_times_per_machine = {machine: 0 for machine in manufacturing_data_sorted['Machine'].unique()}

# Initialize main clock
main_clock = 0

# Initialize variables to track scheduled parts
scheduled_parts = set()

# Schedule manufacturing steps considering sequence and machine availability (FCFS)
for _, row in manufacturing_data_sorted.iterrows():
    part_id = row['Part ID']
    machine = row['Machine']
    cycle_time = row['Cycle Time']
    step = row['Production Step']

    # Find suitable start time for the manufacturing step on the machine
    start_time = max(main_clock, end_times_per_machine[machine])

    # Calculate end time based on start time and cycle time
    end_time = calculate_due_date(start_time, cycle_time)

    # Check if the end time exceeds the total time window
    if end_time > total_time_minutes:
        break

    # Update schedule dictionary with the scheduled manufacturing step
    if machine not in schedule_per_machine:
        schedule_per_machine[machine] = []
    schedule_per_machine[machine].append((part_id, step, start_time, end_time))
    end_times_per_machine[machine] = end_time
    scheduled_parts.add(part_id)

    # Update main clock to the end time of the scheduled step
    main_clock = end_time

# Output the schedule per machine
for machine, schedule in schedule_per_machine.items():
    print(f"Schedule for Machine: {machine}")
    print("{:<15} {:<20} {:<20} {:<10}".format("Part ID", "Production Step", "Start Time", "End Time"))
    for part_id, step, start_time, end_time in schedule:
        print("{:<15} {:<20} {:<20} {:<10}".format(part_id, step, start_time, end_time))
    print()

# Calculate and display capacity utilization for each machine
print("\nCapacity Utilization per Machine:")
for machine, schedule in schedule_per_machine.items():
    machine_capacity = sum(end_time - start_time for _, _, start_time, end_time in schedule)
    utilization = (machine_capacity / total_time_minutes) * 100
    print(f"{machine}, {utilization:.2f}%")

# Calculate total number of parts scheduled and ratio
total_parts = len(manufacturing_data['Part ID'].unique())
scheduled_parts_count = len(scheduled_parts)
ratio = scheduled_parts_count / total_parts * 100 if total_parts != 0 else 0

print(f"\nTotal Parts Scheduled: {scheduled_parts_count}/{total_parts}")
print(f"Ratio of Scheduled Parts: {ratio:.2f}%")


#Longest Processing Time (LPT) Model
The Longest Processing Time (LPT) Model is a scheduling strategy that prioritizes jobs or parts with the longest processing times first. This method aims to optimize machine utilization and reduce the risk of long tasks causing bottlenecks. The following script implements the LPT model to schedule production steps, focusing on maximizing machine capacity utilization and tracking the completion of parts within a defined time window.

**Initialization**

The script mounts Google Drive and loads the manufacturing data.
It defines the time window and other parameters for scheduling.

**Data Preparation**

Manufacturing data is sorted by cycle time in descending order to prioritize longer tasks.

**Scheduling**

Each part's production steps are scheduled on machines based on the LPT model, considering machine availability and sequence.

**Output**

The schedule for each machine is printed, showing the part ID, production step, start time, and end time.

Capacity utilization for each machine is calculated and displayed.
The total number of scheduled parts and the ratio of scheduled parts to total parts are printed.

**Configuration**

Adjust the script as needed to customize the analysis or output formats.

**Contributing**
Contributions are welcome! Please fork the repository, create a new branch, and submit a pull request. For major changes, open an issue first to discuss potential improvements.

**License**

This project is licensed under the MIT License - see the LICENSE file for details. (Annex)

In [ ]:
# LONGEST PROCESSING TIME (LPT) MODEL

import pandas as pd

# Load manufacturing data
manufacturing_data = pd.read_csv('/content/drive/MyDrive/DATA/fertigungsfilm_optimized.csv')

# Define parameters
time_window_days = 10
shifts_per_day = 3
hours_per_shift = 8
minutes_per_shift = hours_per_shift * 60
total_time_minutes = time_window_days * shifts_per_day * minutes_per_shift

# Define due date calculation function
def calculate_due_date(start_time, cycle_time):
    return start_time + cycle_time

# Initialize schedule dictionary to store schedule per machine
schedule_per_machine = {}

# Sort manufacturing data by part ID, production step, and cycle time in descending order of cycle time
manufacturing_data_sorted = manufacturing_data.sort_values(by=['Cycle Time'], ascending=False)

# Initialize dictionary to track end times of manufacturing steps on each machine
end_times_per_machine = {machine: 0 for machine in manufacturing_data_sorted['Machine'].unique()}

# Initialize main clock
main_clock = 0

# Initialize variables to track scheduled parts
scheduled_parts = set()

# Schedule manufacturing steps considering sequence and machine availability (LPT)
for _, row in manufacturing_data_sorted.iterrows():
    part_id = row['Part ID']
    machine = row['Machine']
    cycle_time = row['Cycle Time']
    step = row['Production Step']

    # Find suitable start time for the manufacturing step on the machine
    start_time = max(main_clock, end_times_per_machine[machine])

    # Calculate end time based on start time and cycle time
    end_time = calculate_due_date(start_time, cycle_time)

    # Check if the end time exceeds the total time window
    if end_time > total_time_minutes:
        break

    # Update schedule dictionary with the scheduled manufacturing step
    if machine not in schedule_per_machine:
        schedule_per_machine[machine] = []
    schedule_per_machine[machine].append((part_id, step, start_time, end_time))
    end_times_per_machine[machine] = end_time
    scheduled_parts.add(part_id)

    # Update main clock to the end time of the scheduled step
    main_clock = end_time

# Output the schedule per machine
for machine, schedule in schedule_per_machine.items():
    print(f"Schedule for Machine: {machine}")
    print("{:<15} {:<20} {:<20} {:<10}".format("Part ID", "Production Step", "Start Time", "End Time"))
    for part_id, step, start_time, end_time in schedule:
        print("{:<15} {:<20} {:<20} {:<10}".format(part_id, step, start_time, end_time))
    print()

# Calculate and display capacity utilization for each machine
print("\nCapacity Utilization per Machine:")
for machine, schedule in schedule_per_machine.items():
    machine_capacity = sum(end_time - start_time for _, _, start_time, end_time in schedule)
    utilization = (machine_capacity / total_time_minutes) * 100
    print(f"{machine}, {utilization:.2f}%")

# Calculate total number of parts scheduled and ratio
total_parts = len(manufacturing_data['Part ID'].unique())
scheduled_parts_count = len(scheduled_parts)
ratio = scheduled_parts_count / total_parts * 100 if total_parts != 0 else 0

print(f"\nTotal Parts Scheduled: {scheduled_parts_count}/{total_parts}")
print(f"Ratio of Scheduled Parts: {ratio:.2f}%")


#Slack Time (SLK) Model
The Slack Time (SLK) Model is a scheduling strategy that prioritizes tasks based on the slack time, which is the difference between the time remaining until a task's due date and its processing time. Tasks with the least slack time are prioritized to minimize the risk of missing due dates. The following script implements the SLK model to schedule production steps, focusing on maximizing machine capacity utilization and ensuring timely completion of parts within a defined time window.

**Initialization**

The script loads manufacturing data and defines the time window and other parameters for scheduling.

**Data Preparation**

Manufacturing data is sorted by cycle time in ascending order to prepare for scheduling based on slack time.

**Scheduling**

Each part's production steps are scheduled on machines based on the SLK model, considering machine availability and sequence.

**Output**

The schedule for each machine is printed, showing the part ID, production step, start time, and end time.

Capacity utilization for each machine is calculated and displayed.
The total number of scheduled parts and the ratio of scheduled parts to total parts are printed.

**Configuration**

Adjust the script as needed to customize the analysis or output formats.

**Contributing**
Contributions are welcome! Please fork the repository, create a new branch, and submit a pull request. For major changes, open an issue first to discuss potential improvements.

**License**

This project is licensed under the MIT License - see the LICENSE file for details. (Annex)

In [ ]:
# SLACK TIME (SLK) MODEL

import pandas as pd

# Load manufacturing data
manufacturing_data = pd.read_csv('/content/drive/MyDrive/DATA/fertigungsfilm_optimized.csv')

# Define parameters
time_window_days = 10
shifts_per_day = 3
hours_per_shift = 8
minutes_per_shift = hours_per_shift * 60
total_time_minutes = time_window_days * shifts_per_day * minutes_per_shift

# Define due date calculation function
def calculate_due_date(start_time, cycle_time):
    return start_time + cycle_time

# Initialize schedule dictionary to store schedule per machine
schedule_per_machine = {}

# Sort manufacturing data by part ID, production step, and cycle time in ascending order of cycle time
manufacturing_data_sorted = manufacturing_data.sort_values(by=['Cycle Time'])

# Initialize dictionary to track end times of manufacturing steps on each machine
end_times_per_machine = {machine: 0 for machine in manufacturing_data_sorted['Machine'].unique()}

# Initialize main clock
main_clock = 0

# Initialize variables to track scheduled parts
scheduled_parts = set()

# Schedule manufacturing steps considering sequence and machine availability (SLK)
for _, row in manufacturing_data_sorted.iterrows():
    part_id = row['Part ID']
    machine = row['Machine']
    cycle_time = row['Cycle Time']
    step = row['Production Step']

    # Find suitable start time for the manufacturing step on the machine
    start_time = max(main_clock, end_times_per_machine[machine])

    # Calculate end time based on start time and cycle time
    end_time = calculate_due_date(start_time, cycle_time)

    # Check if the end time exceeds the total time window
    if end_time > total_time_minutes:
        break

    # Update schedule dictionary with the scheduled manufacturing step
    if machine not in schedule_per_machine:
        schedule_per_machine[machine] = []
    schedule_per_machine[machine].append((part_id, step, start_time, end_time))
    end_times_per_machine[machine] = end_time
    scheduled_parts.add(part_id)

    # Update main clock to the end time of the scheduled step
    main_clock = end_time

# Output the schedule per machine
for machine, schedule in schedule_per_machine.items():
    print(f"Schedule for Machine: {machine}")
    print("{:<15} {:<20} {:<20} {:<10}".format("Part ID", "Production Step", "Start Time", "End Time"))
    for part_id, step, start_time, end_time in schedule:
        print("{:<15} {:<20} {:<20} {:<10}".format(part_id, step, start_time, end_time))
    print()

# Calculate and display capacity utilization for each machine
print("\nCapacity Utilization per Machine:")
for machine, schedule in schedule_per_machine.items():
    machine_capacity = sum(end_time - start_time for _, _, start_time, end_time in schedule)
    utilization = (machine_capacity / total_time_minutes) * 100
    print(f"{machine}, {utilization:.2f}%")

# Calculate total number of parts scheduled and ratio
total_parts = len(manufacturing_data['Part ID'].unique())
scheduled_parts_count = len(scheduled_parts)
ratio = scheduled_parts_count / total_parts * 100 if total_parts != 0 else 0

print(f"\nTotal Parts Scheduled: {scheduled_parts_count}/{total_parts}")
print(f"Ratio of Scheduled Parts: {ratio:.2f}%")


#Critical Ration (CR) Model
The Critical Ratio (CR) Model is a scheduling strategy that prioritizes tasks based on the critical ratio, which is the ratio of the time remaining until a task's due date to its processing time. Tasks with the lowest critical ratio are prioritized to ensure timely completion. The following script implements the CR model to schedule production steps, aiming to maximize machine capacity utilization and ensure timely completion of parts within a defined time window.

**Initialization**

The script loads manufacturing data and defines the time window and other parameters for scheduling.

**Data Preparation**

Manufacturing data is sorted by cycle time in ascending order to prepare for scheduling based on critical ratio.

**Scheduling**

Each part's production steps are scheduled on machines based on the CR model, considering machine availability and sequence.
Critical ratio is calculated to prioritize tasks.

**Output**

The schedule for each machine is printed, showing the part ID, production step, start time, end time, and critical ratio.
Capacity utilization for each machine is calculated and displayed.
The total number of scheduled parts and the ratio of scheduled parts to total parts are printed.
The schedule is saved to a CSV file, and a message indicating the file has been saved is printed.

**Configuration**

Adjust the script as needed to customize the analysis or output formats.

**Contributing**
Contributions are welcome! Please fork the repository, create a new branch, and submit a pull request. For major changes, open an issue first to discuss potential improvements.

**License**

This project is licensed under the MIT License - see the LICENSE file for details. (Annex)

In [ ]:
# CRITICAL RATIO (CR) MODEL

import pandas as pd

# Load manufacturing data
manufacturing_data = pd.read_csv('/content/drive/MyDrive/DATA/fertigungsfilm_optimized.csv')

# Define parameters
time_window_days = 10
shifts_per_day = 3
hours_per_shift = 8
minutes_per_shift = hours_per_shift * 60
total_time_minutes = time_window_days * shifts_per_day * minutes_per_shift

# Define due date calculation function
def calculate_due_date(start_time, cycle_time):
    return start_time + cycle_time

# Initialize schedule dictionary to store schedule per machine
schedule_per_machine = {}

# Sort manufacturing data by part ID, production step, and cycle time in ascending order of cycle time
manufacturing_data_sorted = manufacturing_data.sort_values(by=['Cycle Time'])

# Initialize dictionary to track end times of manufacturing steps on each machine
end_times_per_machine = {machine: 0 for machine in manufacturing_data_sorted['Machine'].unique()}

# Initialize main clock
main_clock = 0

# Initialize variables to track scheduled parts
scheduled_parts = set()

# Schedule manufacturing steps considering sequence and machine availability (SLK)
for _, row in manufacturing_data_sorted.iterrows():
    part_id = row['Part ID']
    machine = row['Machine']
    cycle_time = row['Cycle Time']
    step = row['Production Step']

    # Find suitable start time for the manufacturing step on the machine
    start_time = max(main_clock, end_times_per_machine[machine])

    # Calculate end time based on start time and cycle time
    end_time = calculate_due_date(start_time, cycle_time)

    # Check if the end time exceeds the total time window
    if end_time > total_time_minutes:
        break

    # Update schedule dictionary with the scheduled manufacturing step
    if machine not in schedule_per_machine:
        schedule_per_machine[machine] = []
    schedule_per_machine[machine].append((part_id, step, start_time, end_time))
    end_times_per_machine[machine] = end_time
    scheduled_parts.add(part_id)

    # Update main clock to the end time of the scheduled step
    main_clock = end_time

# Output the schedule per machine
for machine, schedule in schedule_per_machine.items():
    print(f"Schedule for Machine: {machine}")
    print("{:<15} {:<20} {:<20} {:<10}".format("Part ID", "Production Step", "Start Time", "End Time"))
    for part_id, step, start_time, end_time in schedule:
        print("{:<15} {:<20} {:<20} {:<10}".format(part_id, step, start_time, end_time))
    print()

# Calculate and display capacity utilization for each machine
print("\nCapacity Utilization per Machine:")
for machine, schedule in schedule_per_machine.items():
    machine_capacity = sum(end_time - start_time for _, _, start_time, end_time in schedule)
    utilization = (machine_capacity / total_time_minutes) * 100
    print(f"{machine}, {utilization:.2f}%")

# Calculate total number of parts scheduled and ratio
total_parts = len(manufacturing_data['Part ID'].unique())
scheduled_parts_count = len(scheduled_parts)
ratio = scheduled_parts_count / total_parts * 100 if total_parts != 0 else 0

print(f"\nTotal Parts Scheduled: {scheduled_parts_count}/{total_parts}")
print(f"Ratio of Scheduled Parts: {ratio:.2f}%")

# Save the schedule to a DataFrame
schedule_data = []
for machine, schedule in schedule_per_machine.items():
    for part_id, step, start_time, end_time in schedule:
        schedule_data.append([machine, part_id, step, start_time, end_time])

schedule_df = pd.DataFrame(schedule_data, columns=['Machine', 'Part ID', 'Production Step', 'Start Time', 'End Time'])

# Define the output CSV file path
output_file = '/content/drive/MyDrive/DATA/fertigungsfilm_CR.csv'

# Save the schedule to a CSV file
schedule_df.to_csv(output_file, index=False)

# Print a message indicating the file has been saved
print(f"Schedule saved to CSV file: {output_file}")


#Multi Machine Operation
The Machine Grouping for Multi Machine Operation process involves categorizing machines into groups and pooling their production steps. This enables optimized scheduling within each group, prioritizing the first machine in the group for task rearrangement. The goal is to enhance machine utilization across the manufacturing process.

**Initialization**

The script loads the manufacturing data and defines machine groups for pooling and optimization.

**Data Pooling**

Production steps are pooled within each machine group, creating schedules for each group based on machine allocation.

**Optimization**

Each group's schedule is sorted by start time.
Tasks within the group are rearranged to prioritize the first machine in the group.
The optimized group schedules are stored for further analysis.

**Machine Utilization Calculation**

For each machine in a group, the script calculates the total operational time and utilized time to determine utilization percentage.

**Output**

The combined optimized schedules are saved to a CSV file.
The script prints a confirmation message and a summary of machine utilizations, showing the utilization percentages for each machine.

**Configuration**

Adjust the script as needed to customize the analysis or output formats.

**Contributing**
Contributions are welcome! Please fork the repository, create a new branch, and submit a pull request. For major changes, open an issue first to discuss potential improvements.

**License**

This project is licensed under the MIT License - see the LICENSE file for details. (Annex)

In [ ]:
# MACHINE GROUPING FOR MULTI MACHINE OPERATION

import pandas as pd

# Load manufacturing data
manufacturing_data = pd.read_csv('/content/drive/MyDrive/DATA/fertigungsfilm_CR.csv')

# Define machine groups
machine_groups = {
    1: ['machine_1', 'machine_2', 'machine_3'],
    2: ['machine_4', 'machine_5'],
    3: ['machine_6'],
    4: ['machine_7', 'machine_8'],
    5: ['machine_9', 'machine_10', 'machine_11']
}

# Pool production steps within each machine group
pooled_schedule = {}
for group, machines in machine_groups.items():
    group_schedule = manufacturing_data[manufacturing_data['Machine'].isin(machines)]
    pooled_schedule[group] = group_schedule

# Optimize scheduling for each machine group
optimized_schedule = {}
machine_utilizations = []

for group, schedule in pooled_schedule.items():
    # Sort schedule by start time
    schedule = schedule.sort_values(by='Start Time')
    # Rearrange tasks within the group schedule with the first machine prioritized
    first_machine_schedule = schedule[schedule['Machine'] == machine_groups[group][0]]
    other_machines_schedule = schedule[schedule['Machine'] != machine_groups[group][0]]
    optimized_group_schedule = pd.concat([first_machine_schedule, other_machines_schedule])
    optimized_schedule[group] = optimized_group_schedule

    # Calculate machine utilizations for each machine in the group
    for machine in machine_groups[group]:
        machine_schedule = optimized_group_schedule[optimized_group_schedule['Machine'] == machine]
        total_time = machine_schedule['End Time'].max() - machine_schedule['Start Time'].min()
        utilized_time = machine_schedule['End Time'].sum() - machine_schedule['Start Time'].sum()
        utilization_percentage = (utilized_time / total_time) * 100
        machine_utilizations.append((machine, utilization_percentage))

# Combine all optimized schedules into one DataFrame
combined_optimized_schedule = pd.concat(optimized_schedule.values())

# Define the path for the output CSV file
output_csv_path = '/content/drive/MyDrive/DATA/fertigungsfilm_CR_optimized.csv'

# Export the combined optimized schedule to CSV
combined_optimized_schedule.to_csv(output_csv_path, index=False)

# Output confirmation message
print(f"Optimized schedule saved to: {output_csv_path}")

# Summary of machine utilizations
print("\nSummary of Machine Utilizations:")
print("{:<15} {:<10}".format("Machine", "Utilization"))
for machine, utilization in machine_utilizations:
    print("{:<15} {:<10.2f}%".format(machine, utilization))


#Heuristic Modeling
Heuristic modeling is a method used to solve complex scheduling problems by following a rule-based approach. In this script, we aim to optimize the scheduling of manufacturing tasks across multiple machines while ensuring sequencing constraints are met. The goal is to maximize machine utilization and ensure as many parts as possible are scheduled within a given time window.

**Loading and Sorting Data**

The manufacturing data is loaded from a CSV file and sorted by Part ID and Production Step to maintain the sequence of operations.

**Initialization**

Dictionaries are initialized to keep track of schedules and end times for each machine, as well as the end times for each part's production steps.

**Scheduling Logic**

The script iterates through each part and its production steps, assigning them to machines while respecting sequencing constraints. The start and end times for each step are calculated based on the maximum of the current time, the part's end time, and the machine's end time to avoid overlaps.

**Parameters for Scheduling**

The script defines a time window of 10 days, with 3 shifts per day and 8 hours per shift, to calculate the total available time in minutes.

**Calculating Utilization and Scheduling Ratios**

The total number of unique parts and the number of parts successfully scheduled are calculated. Machine utilization rates are determined by comparing the utilized time to the total available time.

**Output**

The script prints the utilization rates for each machine and the ratio of scheduled parts to total parts. It also provides the detailed schedule for each machine, showing the Part ID, Production Step, Start Time, and End Time.

**Configuration**

Adjust the script as needed to customize the analysis or output formats.

**Contributing**
Contributions are welcome! Please fork the repository, create a new branch, and submit a pull request. For major changes, open an issue first to discuss potential improvements.

**License**

This project is licensed under the MIT License - see the LICENSE file for details. (Annex)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# HEURISTIC MODELING

import pandas as pd

# Load manufacturing data from CSV
file_path = '/content/drive/MyDrive/DATA/fertigungsfilm.csv'
manufacturing_data = pd.read_csv(file_path)

# Sort data by Part ID and Production Step for sequencing
manufacturing_data.sort_values(['Part ID', 'Production Step'], inplace=True)

# Initialize a schedule dictionary to track assigned production steps for each machine
machine_schedules = {f"machine_{i}": [] for i in range(1, 12)}  # Assuming 11 machines

# Track the end time for each machine to enforce sequencing and avoid overlaps
machine_end_times = {f"machine_{i}": 0 for i in range(1, 12)}

# Dictionary to keep track of the last end time for each part's production steps
part_end_times = {}

# Iterate through each part and assign production steps to machines with sequencing
for part_id, part_data in manufacturing_data.groupby('Part ID'):
    current_time = 0  # Start time for the first production step of each part
    part_end_times[part_id] = 0  # Initialize the end time for this part

    for index, row in part_data.iterrows():
        production_step = row['Production Step']
        machine = row['Machine']
        cycle_time = row['Cycle Time']

        # Calculate start time for the production step with sequencing
        start_time = max(current_time, part_end_times[part_id], machine_end_times[machine])
        end_time = start_time + cycle_time

        # Assign the production step to the machine in the schedule
        machine_schedules[machine].append((part_id, production_step, start_time, end_time))

        # Update end time for the part and the machine to enforce sequencing
        part_end_times[part_id] = end_time
        machine_end_times[machine] = end_time

# Define parameters for scheduling
time_window_days = 10
shifts_per_day = 3
hours_per_shift = 8
minutes_per_shift = hours_per_shift * 60
total_time_minutes = time_window_days * shifts_per_day * minutes_per_shift

# Calculate total number of unique parts in the source file
total_parts = len(set(manufacturing_data['Part ID']))

# Calculate the number of unique parts that were successfully scheduled
scheduled_parts = len(set(part_end_times.keys()))

# Calculate machine utilization rates and summarize results
print("\nMachine Utilization Rates:")
for machine, schedule in machine_schedules.items():
    utilization_time = sum(end_time - start_time for _, _, start_time, end_time in schedule)
    utilization_rate = utilization_time / total_time_minutes
    print(f"Machine {machine}: Utilization Rate = {utilization_rate:.2%}")

scheduled_ratio = scheduled_parts / total_parts
print(f"\nScheduled Parts Ratio: {scheduled_parts} out of {total_parts} parts ({scheduled_ratio:.2%}) scheduled.")

# Print the best schedule for each machine
print("\nBest Schedule for Each Machine:")
for machine, schedule in machine_schedules.items():
    print(f"\nMachine {machine}:")
    print("{:<15} {:<20} {:<20} {:<10}".format("Part ID", "Production Step", "Start Time", "End Time"))
    for part_id, step, start_time, end_time in schedule:
        print("{:<15} {:<20} {:<20} {:<10}".format(part_id, step, start_time, end_time))


#Annex

In [ ]:
## License

This project is licensed under the MIT License - see the [LICENSE](LICENSE) file for details. The MIT License is a permissive license that allows you to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the software, with proper attribution and without warranty.


In [ ]:
# MIT License

# Copyright (c) [2024] [Lean-IQ, Ralf Puehler]

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
#FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.